In [ ]:
from Params import DataParams, NetworkParams, TrainingParams, Params
from data_helper import XORDataset, getVariableLengths
from Model import Model, train_model
import torch
import numpy as np

In [ ]:
def generateDataParams(max_seq_len, min_seq_len=5):
    def num_samples_fn(seq_len):
        max_seqs = 100000
        num_seqs = seq_len ** 2
        return min(num_seqs, max_seqs)
    data_params_list = []
    for seq_len in range(min_seq_len, max_seq_len+1):
        num_samples = num_samples_fn(seq_len)
        data_params_list.append(DataParams(num_samples, seq_len, False))
        data_params_list.append(DataParams(num_samples, seq_len, True))
    return data_params_list

In [ ]:
max_seq_len = 50
data_params_list = generateDataParams(max_seq_len)
network_params = NetworkParams()
training_params = TrainingParams()
num_runs_per_model = 50

In [ ]:
num_steps_varying_seq = dict()
num_steps_fixed_seq = dict()
for data_params in data_params_list:
    params = Params(data_params, network_params, training_params)
    num_steps_avg = 0
    for num_run in range(num_runs_per_model):
        model = Model(network_params)
        seed = data_params.max_seq_len * num_run
        torch.manual_seed(seed)
        np.random.seed(seed)
        num_steps = train_model(model, params)
        num_steps_avg += num_steps
    num_steps_avg /= num_runs_per_model
    print(f"Average number of steps:{num_steps_avg}")
    if data_params.is_seq_len_varying:
        num_steps_varying_seq[data_params.max_seq_len] = num_steps_avg
    else:
        num_steps_fixed_seq[data_params.max_seq_len] = num_steps_avg